<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/Video_ViT_SWIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 35.1 MB/s eta 0:00:00


In [ ]:
import os

# Set a custom Hugging Face home directory
os.environ['HF_HOME'] = '/content'

Video Classification (deafult 32 frames)

In [ ]:
import av
import numpy as np
import torch

from transformers import VivitImageProcessor, VivitForVideoClassification
from huggingface_hub import hf_hub_download

np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

# sample 32 frames
indices = sample_frame_indices(clip_len=32, frame_sample_rate=4, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container=container, indices=indices)

image_processor = VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400")
model = VivitForVideoClassification.from_pretrained("google/vivit-b-16x2-kinetics400")

inputs = image_processor(list(video), return_tensors="pt")
print('input video:', inputs['pixel_values'].shape)
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# model predicts one of the 400 Kinetics-400 classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])
print('Predicted Label:', predicted_label)

eating_spaghetti.mp4:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/18.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/356M [00:00<?, ?B/s]

input video: torch.Size([1, 32, 3, 224, 224])
LABEL_116
Predicted Label: 116


Video Classification (customized 5 frames)

In [ ]:
from transformers import VivitImageProcessor, VivitForVideoClassification, VivitConfig
from safetensors.torch import load_file
from transformers import AutoConfig

from collections import OrderedDict


def partial_loading(model, state_dict):
    new_state_dict = OrderedDict()

    for k, v in state_dict.items():
        if k in model.state_dict() and model.state_dict()[k].shape == v.shape:
            new_state_dict[k] = v

    model.load_state_dict(new_state_dict, strict=False)
    return model

# sample 5 frames
num_frames = 5
indices = sample_frame_indices(clip_len=5, frame_sample_rate=4, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container=container, indices=indices)

image_processor = VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400")
# model = VivitForVideoClassification.from_pretrained("google/vivit-b-16x2-kinetics400")


config = AutoConfig.from_pretrained("google/vivit-b-16x2-kinetics400")
config.num_frames = num_frames
# config.video_size = [num_frames, 224, 224] #518, 644
config.video_size = [num_frames, 518, 644]

model = VivitForVideoClassification(config)
model_weight = '/content/hub/models--google--vivit-b-16x2-kinetics400/snapshots/8a7171a57f79b9aaa58bc8d977c002a0ea0f0d42/pytorch_model.bin'
state_dict = torch.load(model_weight, weights_only=True)
model = partial_loading(model, state_dict)



inputs = image_processor(list(video), return_tensors="pt")
print('input video:', inputs['pixel_values'].shape)
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# model predicts one of the 400 Kinetics-400 classes
predicted_label = logits.argmax(-1).item()
# print(model.config.id2label[predicted_label])
print('Predicted Label:', predicted_label)

input video: torch.Size([1, 5, 3, 224, 224])
Predicted Label: 187


In [ ]:
emb = model.vivit.embeddings
out = emb(inputs['pixel_values'])
print(out.shape)

torch.Size([1, 393, 768])


Video Swin: https://github.com/haofanwang/video-swin-transformer-pytorch

In [ ]:
!git clone https://github.com/haofanwang/video-swin-transformer-pytorch.git
%cd video-swin-transformer-pytorch
!mkdir checkpoints && cd checkpoints
!wget https://github.com/SwinTransformer/storage/releases/download/v1.0.4/swin_base_patch244_window1677_sthv2.pth
%cd ..


Cloning into 'video-swin-transformer-pytorch'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 54 (delta 19), reused 14 (delta 14), pack-reused 30 (from 1)
Receiving objects: 100% (54/54), 21.95 KiB | 2.74 MiB/s, done.
Resolving deltas: 100% (22/22), done.
--2025-01-10 17:15:09--  https://github.com/SwinTransformer/storage/releases/download/v1.0.4/swin_base_patch244_window1677_sthv2.pth
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/357198522/20458080-d55e-11eb-9021-4730e624e0ea?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250110%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250110T171509Z&X-Amz-Expires=300&X-Amz-Signature=6455db2ec5ae41f8c30df14d20

In [ ]:
!pip -q install mmcv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.1/479.1 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.3/452.3 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 17.2 MB/s eta 0:00:00


In [ ]:
%cd /content/video-swin-transformer-pytorch
import torch
import torch.nn as nn
from video_swin_transformer import SwinTransformer3D

model = SwinTransformer3D()
print(model)

dummy_x = torch.rand(1, 3, 32, 224, 224)
logits = model(dummy_x)
print(logits.shape)

In [ ]:
import torch
import torch.nn as nn
from collections import OrderedDict
from video_swin_transformer import SwinTransformer3D

model = SwinTransformer3D(embed_dim=128,
                          depths=[2, 2, 18, 2],
                          num_heads=[4, 8, 16, 32],
                          patch_size=(2,4,4),
                          window_size=(16,7,7),
                          drop_path_rate=0.4,
                          patch_norm=True)

# https://github.com/SwinTransformer/Video-Swin-Transformer/blob/master/configs/recognition/swin/swin_base_patch244_window1677_sthv2.py
checkpoint = torch.load('./checkpoints/swin_base_patch244_window1677_sthv2.pth')

new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    if 'backbone' in k:
        name = k[9:]
        new_state_dict[name] = v

model.load_state_dict(new_state_dict)

dummy_x = torch.rand(1, 3, 32, 224, 224)
logits = model(dummy_x)
print(logits.shape)